In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Loading the data 
## Loading Train Data
train_subset = pd.read_csv("/content/drive/MyDrive/FinalHack Datasets/Year-Wise /train_2018.csv")
## Loading Test Data
test = pd.read_csv("/content/drive/MyDrive/FinalHack Datasets/test.csv")
## Loading Item Details Data 
item_details = pd.read_excel("/content/drive/MyDrive/FinalHack Datasets/item_details.xlsx")

In [ ]:
train_subset['date'] = pd.to_datetime(train_subset['date'])

train_subset['Week'] =pd.DatetimeIndex(train_subset['date']).weekday.astype('int8')

train_subset = pd.merge(train_subset,item_details,how = 'left', on = 'item_id')

train_subset = train_subset.drop(['Unnamed: 0','date','Year','perishable'],axis = 1)

train_subset.head()

,Unnamed: 0,date,locationId,item_id,unit_sales,onpromotion,Year,Month,Day,Week,category_of_item,class,perishable
0,66458908,2018-01-01,location_25,item_105574,12.0,False,2018,1,1,0,grocery_items,class_1045,0
1,66458909,2018-01-01,location_25,item_105575,9.0,False,2018,1,1,0,grocery_items,class_1045,0
2,66458910,2018-01-01,location_25,item_105857,3.0,False,2018,1,1,0,grocery_items,class_1092,0
3,66458911,2018-01-01,location_25,item_108634,3.0,False,2018,1,1,0,grocery_items,class_1075,0
4,66458912,2018-01-01,location_25,item_108701,2.0,True,2018,1,1,0,deli_items,class_2644,1


In [ ]:
train_subset = train_subset.drop(['Unnamed: 0','date','Year','perishable'],axis = 1)

In [ ]:
train_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35229871 entries, 0 to 35229870
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   locationId        object 
 1   item_id           object 
 2   unit_sales        float64
 3   onpromotion       bool   
 4   Month             int64  
 5   Day               int64  
 6   Week              int64  
 7   category_of_item  object 
 8   class             object 
dtypes: bool(1), float64(1), int64(3), object(4)
memory usage: 2.4+ GB


In [ ]:
### Removing negative values in train data 

train_subset = train_subset[(train_subset['unit_sales']>0)]

In [ ]:
### Removing outliers 

Q1 = train_subset.unit_sales.quantile(0.25)
Q3 = train_subset.unit_sales.quantile(0.75)
print(Q1,Q3)


IQR = Q3 - Q1
print(IQR)

lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR
print( lower_limit,upper_limit)


train_subset = train_subset[(train_subset.unit_sales < upper_limit)]

2.0 8.0
6.0
-7.0 17.0


In [ ]:
## Label Encoding 

from sklearn import preprocessing
def df_lbl_enc(df):
    for c in df.columns:
        if df[c].dtype == 'object':
            lbl = preprocessing.LabelEncoder()
            df[c] = lbl.fit_transform(df[c])
            print(c)
    return df


train_subset = df_lbl_enc(train_subset)

locationId
item_id
category_of_item
class


In [ ]:
 from sklearn.preprocessing import LabelEncoder

 lb = LabelEncoder()

 train_subset['onpromotion']= lb.fit_transform(train_subset['onpromotion'])

In [ ]:
### Converting to category 


def cat_converter(df):
    for i in df.columns:
        if df[i].dtype == 'float64':
            df[i] = df[i]
        else:            
            df[i] = df[i].astype('category')
            print(i)
    return df


train_subset = cat_converter(train_subset)

locationId
item_id
onpromotion
Month
Day
Week
category_of_item
class


In [ ]:
cat_cols = ['locationId','item_id','onpromotion','category_of_item','class','Month','Day','Week']

In [ ]:
from sklearn.model_selection import train_test_split

X = train_subset.drop(['unit_sales'], axis = 1)

y = train_subset['unit_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [ ]:
from lightgbm import LGBMRegressor 


## Light GBM 

LGB_model = LGBMRegressor()


LGB_model.fit(X_train, y_train_log,categorical_feature = cat_cols)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
LGB_Model_Y_train_pred = LGB_model.predict(X_train)
LGB_Model_Y_test_pred = LGB_model.predict(X_test)


LGB_Model_Y_train_pred =np.expm1(LGB_Model_Y_train_pred)
LGB_Model_Y_test_pred =np.expm1(LGB_Model_Y_test_pred)

Train_score_LGB= mean_absolute_percentage_error(y_train,LGB_Model_Y_train_pred)
Test_score_LGB = mean_absolute_percentage_error(y_test,LGB_Model_Y_test_pred)

print(Train_score_LGB)
print(Test_score_LGB)

72.61493477925214
72.67050868401846
